In [11]:
import gymnasium as gym
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.animation import FuncAnimation

MAX_MISSILES = 15

"""
The point class doesn't create missiles, it creates an attack, defense, or target point (these are in turn used to create missiles).
"""

class Point: 
    def __init__(self, type):
        self.type = type
        self.coordinates = np.array([0, 0])
        self.radius = 0.2

    def create_target(self):
        x = random.uniform(-3.0, 3.0)
        y = random.uniform(-2.0, 2.0)
        self.coordinates = np.array([x, y])
        self.theta = 0 # -------------------- DON'T SEE WHERE THIS IS USED
        return self.coordinates, self.theta # ----------------- DON'T SEE HOW THETA IS USED
    
    def create_defense(self, target):
        x = random.uniform((target.coordinates[0] - self.radius) - 1.5, (target.coordinates[0] + self.radius) + 1.5)
        y = random.uniform((target.coordinates[1] - self.radius) - 1.5, (target.coordinates[1] + self.radius) + 1.5)
        self.coordinates = np.array([x, y])
        return self.coordinates

    def create_attack(self, target):
        x_side_left = random.uniform(-9.5, (target.coordinates[0] - self.radius) - 2)
        x_side_right = random.uniform((target.coordinates[0] + self.radius) + 2, 9.5)
        y_below = random.uniform((target.coordinates[1] - self.radius) - 2, -9.5)
        y_above = random.uniform((target.coordinates[1] + self.radius) + 2, 9.5)
        x_inclusive = random.uniform(-9.5, 9.5)
        y_inclusive = random.uniform(-9.5, 9.5)
        y_below_x_inclusive = np.array([x_inclusive, y_below])
        y_above_x_inclusive = np.array([x_inclusive, y_above])
        x_left_y_inclusive = np.array([x_side_left, y_inclusive])
        x_right_y_inclusive = np.array([x_side_right, y_inclusive])

        self.coordinates = random.choice([y_below_x_inclusive, y_above_x_inclusive, x_left_y_inclusive, x_right_y_inclusive])
        
        # Create attack_theta, .attack_theta_rad allows us to aim towards the target
        self.AT_abs_length = math.hypot(self.coordinates[0] - target.coordinates[0], self.coordinates[1] - target.coordinates[1])
        self.length_adjacent_r = abs(target.coordinates[0] - self.coordinates[0])
        self.attack_theta_rad = math.acos(self.length_adjacent_r / self.AT_abs_length)

        if self.coordinates[0] < target.coordinates[0]:
            if self.coordinates[1] < target.coordinates[1]:
                self.attack_theta_rad = self.attack_theta_rad
            else:
                self.attack_theta_rad = ((2 * math.pi) - self.attack_theta_rad)
            
        else:
            if self.coordinates[1] < target.coordinates[1]:
                self.attack_theta_rad = ((math.pi) - self.attack_theta_rad)
            else:
                self.attack_theta_rad = ((math.pi) + self.attack_theta_rad)

        return self.coordinates, self.attack_theta_rad

class Missile:
    def __init__(self, type):
        self.type = type
        self.coordinates = np.array([0, 0])
        self.radius = 0.2 # ----------------- I THINK THIS IS SPECIFIED IN THE POINT CLASS
    
    # Not sure this code is being used to create missiles, or the version specified below
    def fire_attack_missile(self, attack):
        self.speed = 1
        self.theta = attack.attack_theta_rad
        self.coordinates = attack.coordinates
        self.fired = True
        # self.fired_time = global_time # ------------------ ALSO NOT SURE THIS IS USED

        return self.coordinates, self.theta, self.speed, self.fired
    
    # def fire_attack_missile(self, global_time):
    #     self.fired = True
    #     self.fired_time = global_time
    #     return self.fired, self.fired_time
    
    # This one as well
    def create_defense_missile(self, defense, theta):
        self.speed = 1
        self.theta = theta
        self.coordinates = defense.coordinates
        self.fired = False
        self.fired_time = 0 
        self.defense_missile_to_attack_missile = None
        self.start_distance = None
        self.reward_diff = None

    def fire_defense_missile(self, global_time):
        self.fired = True
        self.fired_time = global_time
        self.initial_position = np.array([self.coordinates[0], self.coordinates[1]])
        return self.fired, self.fired_time

class Environment:
    def __init__(self):
        self.global_time = 0
        self.reward = 0
        self.done = False

        self.target = None
        self.defense = None
        self.attack = None
        self.attack_missile = None
        self.initial_defense_missile = None
        self.defense_missiles = []
        self.attack_missile_to_target = None

    def set_episode(self, global_time):
        self.reward = 0
        self.target = Point('target')
        self.defense = Point('defense')
        self.attack = Point('attack')

        self.target.create_target()
        self.defense.create_defense(self.target)
        self.attack.create_attack(self.target)

        self.attack_missile = Missile('attack')
        self.attack_missile.fire_attack_missile(self.attack)
        self.attack_missile_to_target = self.distance_attack_to_target(self.attack_missile)

        self.defense_missiles = []
        self.initial_defense_missile = Missile('defense')
        self.intial_defense_missile.create_defense_missile(self.defense, 0)
        self.defense_missiles.append(self.initial_defense_missile)

    def local_create_defense_missile(self, defense, global_time, theta):
        defense_missile = Missile('defense')
        defense_missile.create_defense_missile(defense, global_time, theta)
        self.defense_missiles.append(defense_missile)
        return defense_missile
    
    def calculate_distance(self, point1, point2):
        return math.hypot(point1[0] - point2[0], point1[1] - point2[1])

    def distance_attack_to_target(self, attack_missile):
        self.attack_missile_to_target = (self.calculate_distance(attack_missile.coordinates, self.target.coordinates) - (attack_missile.radius + self.target.radius))
        if self.attack_missile_to_target < 0:
            self.reward += -100
        else:
            print("BEFORE REWARD", self.reward)
            print("DISTANCE ATTACK TO TARGET", self.attack_missile_to_target)
            print("UNO SOBRE DISTANCE ATTACK TO TARGET", 1 / self.attack_missile_to_target)
            print("DOS VECES ESA MIERDA", -2 * (1 / self.attack_missile_to_target))
            print("LA MIERDA QUE ESTAMOS RESTANDO --------------------------------", -2 * (1 / self.attack_missile_to_target))
            self.reward += (-2 * (1 / self.attack_missile_to_target))

        print("AFTER REWARD", self.reward)
        return self.attack_missile_to_target


env = Environment()
env.set_episode(0)










BEFORE REWARD 0
DISTANCE ATTACK TO TARGET 7.150868464856584
UNO SOBRE DISTANCE ATTACK TO TARGET 0.13984315400493885
DOS VECES ESA MIERDA -0.2796863080098777
LA MIERDA QUE ESTAMOS RESTANDO -------------------------------- -0.2796863080098777
AFTER REWARD -0.2796863080098777


AttributeError: 'Environment' object has no attribute 'intial_defense_missile'